In [8]:
import tensorflow as tf
tf.__version__

'1.12.0'

In [9]:
tf.keras.__version__

'2.1.6-tf'

# 初识神经网络
本笔记本包含<a target='_blank' href='https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff'>Python深度学习</a>的第2章第1节中的代码示例。请注意，原始文本包含更多的内容，特别是进一步的解释和相关实验图像的绘制:而在这个笔记本中，您只会找到源代码和相关的注释。

现在我们来看第一个神经网络的具体例子，它使用Python库Keras来学习对手写数字进行分类(PS:由于tensorFlow从1.9版开始，直接增加了kares模块，因此后续内容中直接从TensorFlow中使用keras，以保持和TensorFlow的兼容)。除非您已经有了使用Keras或类似库的经验，否则您不会马上理解第一个示例的所有内容。您可能还没有安装Keras。别担心。在下一章中，我们将回顾示例中的每个元素并详细解释它们。所以，如果有些步骤看起来很随意或很神奇，不要担心!我们总得找个地方开始。

我们试图解决的问题是对灰度图像进行分类的手写数字(由28像素\*28像素组成),10个类别(0到9)。我们将使用的数据集是MNIST数据集,一个在机器学习社区中典型的数据集,该数据集基本上从机器学习出现以来就同时存在了。因此其本身已经被人们非常深入的研究过。该数据集包含60000张训练图像，已经10000张测试图像，由国家标准与技术研究所(简称NIST)在20世纪80年代收集。您可以将“求解”MNIST看作是深度学习的“Hello World”一样的入门程序——您要做的就是验证您的算法是否如预期的那样工作。当您以后再实践机器学习算法时，将会看到MNIST在科学论文、博客文章等中反复出现。

MNIST数据集以Keras的形式预先加载，以一组四维Numpy数组的形式存储：

In [11]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

`train_images`以及`train_labels`来自于“训练集”，即模型将要学习的资料(PS:可以理解为高中参加高考前平常学习使用的大量教科书)。然后模型将要被“测试集”测试(可以理解为平常每周测试用的试卷)，即`test_images`和`test_labels`。我们的图像被编码为Numpy数组，标签只是一个数字数组，范围从0到9。图像和标签之间存在一一对应关系。

让我们来瞧一瞧训练数据:

In [12]:
train_images.shape

(60000, 28, 28)

In [13]:
len(train_labels)

60000

In [14]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

让我们继续观察一下测试数据：

In [15]:
test_images.shape

(10000, 28, 28)

In [16]:
len(test_labels)

10000

In [17]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

我们的工作流程如下:
- 首先，我们将用训练数据、`train_images`和`train_label`输入我们的神经网络
- 然后，该网络将学习关联图像和其对应的标签
- 最后，我们将要求网络为`test_images`生成预测标签，并验证这些预测是否与`test_labels`中的标签匹配

让我们来构建我们的网络——同样强调，请记住，您暂时不需要完全理解这个示例。

In [18]:
from tensorflow.keras import models
from tensorflow.keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

神经网络的核心构建模块是“层”，一个数据处理模块，你可以把它想象成数据的“过滤器”。一些数据进来，然后以一种更有用的形式出来。准确地说，层从提供给它们的数据中提取表示——希望这些表示对当前的问题更有意义。大多数深度学习实际上是将简单的层链接在一起，从而实现一种渐进的“数据蒸馏”形式。深度学习模型就像一个数据处理的筛子，由一系列不断改进的数据过滤器(即“层”)组成。

在这里，我们的网络由两个`dense`层组成，它们是紧密连接(也称为“完全连接”)的神经层。第二个(也是最后一个)层是一个10通道“softmax”层，这意味着它将返回一个包含10个概率值的数组(总和为1)。

为了使我们的网络为训练做好准备，我们需要选择另外三件事，作为“编译”步骤的一部分:
- 损失函数:这是网络如何能够衡量它在培训数据上做得有多好，从而如何能够将自己引导到正确的方向
- 优化器:这是一种机制，通过这种机制，网络将根据所看到的数据及其损失函数更新自己
- 在训练和测试期间要监控的指标。在这里，我们只关心精确度(图像被正确分类的部分)

损失函数和优化器的详细用途将在接下来的两章中阐明。

In [20]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

在训练之前，我们将对数据进行预处理，将其重塑为网络期望的形状，并对其进行缩放，使所有值都在`[0,1]`区间内。例如，以前我们的训练图像存储在`uint8`类型的形状数组`(60000,28,28)`中，值的范围是[0,255]。我们将其转换为一个类型为`float32`形状数组是`(60000,28 * 28)`的数据，其值介于0和1之间。

In [21]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

我们还需要对标签进行分类编码，这一步我们将在第三章中解释:

In [23]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

我们现在已经准备好训练我们的网络，在Keras中，这是通过调用网络的fit方法来完成的:我们将模型“学习”它的训练数据。

In [25]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 4s 65us/step - loss: 0.2544 - acc: 0.9277
Epoch 2/5
60000/60000 [==============================] - 4s 68us/step - loss: 0.1049 - acc: 0.9687
Epoch 3/5
60000/60000 [==============================] - 4s 69us/step - loss: 0.0692 - acc: 0.9792
Epoch 4/5
60000/60000 [==============================] - 4s 65us/step - loss: 0.0499 - acc: 0.9852
Epoch 5/5
60000/60000 [==============================] - 4s 61us/step - loss: 0.0371 - acc: 0.9890


训练时显示两个量:网络对训练数据的“损失”量和网络对训练数据的准确性度量。我们对训练数据的准确率很快达到了0.989(即98.9%)。现在让我们检查我们的模型在测试集上的表现是否也很好。

In [26]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

10000/10000 [==============================] - 0s 47us/step


In [27]:
print('test_acc:', test_acc)

test_acc: 0.9779


我们的测试集精度是97.8%，这比训练集的精度要低很多。训练精度和测试精度之间的差距是“过拟合”的一个例子，即机器学习模型在新数据上的表现往往比在它们的训练数据上差。过度拟合将是第三章的中心主题。

这就结束了我们的第一个示例——您刚刚看到了我们如何构建和训练一个神经网络来对手写数字进行分类，这只用了不到20行Python代码。在下一章中，我们将详细介绍我们刚刚预览过的每一个动态片段，并阐明幕后的真实情况。你将学习“张量”，进入网络的数据存储对象，张量运算，由哪些层组成，以及梯度下降，它允许我们的网络从它的训练例子中学习。